In [15]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from lxml import html
import scrapy
from time import sleep
import urllib3
import json
from selenium import webdriver
import random

In [16]:
def parser(link):

    encabezados = {
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36'
    }

    resp = requests.get(link, headers=encabezados, verify=False)
    resp = resp.text

    #soup = get_Soup('https://minciencias.gov.co/convocatorias/todas')
    parser = html.fromstring(resp)
    urllib3.disable_warnings()

    return parser

In [17]:
#links
estados = ['https://www.conicyt.cl/blog/category/concursos/abiertos/', 'https://www.conicyt.cl/blog/category/concursos/en-evaluacion/', 'https://www.conicyt.cl/blog/category/concursos/adjudicados/', 'https://www.conicyt.cl/blog/category/concursos/proximos/']

def links_proyectos(estados_chile):

    links = []
    for estado in estados:

        parserT = parser(estado)

        #Extrae y organiza lista de paginacion
        paginacion = parserT.xpath('//div[@class="pagenavi"]/ul//a/@href')
        
        urls = []
        pagina = 'https://www.conicyt.cl/blog/category/concursos/adjudicados/'
        if len(paginacion) != 0:
            paginacion.pop()
            paginacion.append('?pag=1')
            for link in paginacion:
                urls.append(pagina+link)
        else:
            urls.append(estado)
        
        

        #Extrae links de cada pag
        
        for url in urls:
            parser_url = parser(url)
            links_pagina = parser_url.xpath('//div[@id="contenido_doble"]//h2/a/@href')
            links += links_pagina
        
        
        for link in links:
            if link.startswith('https://www.conicyt.cl/becasconicyt'):
                links.remove(link)


    return links

urls_proyectos = links_proyectos(estados)



In [32]:
### avance chile
chile = pd.DataFrame()

descripciones = []
publicos = []
bitacoras = []
links = []
titulos = []

descripcion = ''
p_objetivo = ''
bitacora = ''
titulo = ''
link = ''
for url_proy in urls_proyectos[0:5]:

    driver = webdriver.Firefox()
    driver.get(url_proy)

    #Saca Titulo
        
    try:
        titulo = driver.find_element_by_xpath('//h2[@id="title-concurso"]').text
        titulos.append(titulo)
    except:
        
        try:
            if len(titulo) == 0:
                titulo = driver.find_element_by_xpath('//h2[@class="title_content"]').text
                titulos.append(titulo)
        except:        
            titulo = ''
            titulos.append(titulo)


    ### Saca el formato descripción

    try:
        boton_descripcion = driver.find_element_by_xpath('//div[contains(text(), "presentación")]')
            
    except:
        try:
            boton_descripcion = driver.find_element_by_xpath('//li[contains(text(), "Presentación")]')
        except:
            try:
                boton_descripcion = driver.find_element_by_xpath('//div[contains(@class, "accordion-Presentacin")]')
            except:
                descripcion = ''
                descripciones.append(descripcion)
                continue
                
                
    boton_descripcion.click()

    try:
        descripcion = driver.find_element_by_xpath('.//div[@id="content-1"]').text
        descripciones.append(descripcion)
    except:
        try:
            descripcion = driver.find_element_by_xpath('.//div[@id="pane-0-0"]').text
            descripciones.append(descripcion)
        except:
            descripcion = driver.find_element_by_xpath('.//div[@class="arconix-accordion-content"]').text
            descripciones.append(descripcion)



    ### Saca el publico objetivo

   
    try:
        boton_p_objetivo = driver.find_element_by_xpath('//div[contains(text(), "público")]')
        
    except:
        try:
            boton_p_objetivo = driver.find_element_by_xpath('//div[contains(text(), "Público")]')
        except:
            try:
                boton_p_objetivo = driver.find_element_by_xpath('//ul//li[contains(text(), "Público")]') 
            except:
                p_objetivo = ''
                publicos.append(p_objetivo)
                continue
                
            
    boton_p_objetivo.click()
    try:
        p_objetivo = driver.find_element_by_xpath('.//div[@id="content-2"]').text
        publicos.append(p_objetivo)
    except:
        p_objetivo = driver.find_element_by_xpath('.//div[@id="pane-0-1"]').text
        publicos.append(p_objetivo)

    
    ### Saca Bitacoras
    try:
        boton_bitacora = driver.find_element_by_xpath('//div[contains(text(), "bitacora")]')
    except:
        try:
            boton_bitacora = driver.find_element_by_xpath('//div[contains(text(), "bitácora")]')
        except:
            try:
                boton_bitacora = driver.find_element_by_xpath('//div[contains(text(), "Bitácora")]')
            except:
                try:
                    boton_bitacora = driver.find_element_by_xpath('//div[contains(text(), "Bitacora")]')
                except:
                    try:
                        boton_bitacora = driver.find_element_by_xpath('//ul//li[contains(text(), "bitacora")]') 
                    except:
                        try:
                            boton_bitacora = driver.find_element_by_xpath('//li[contains(text(), "Bitácora")]')
                        except:
                            bitacora = ''
                            bitacoras.append(bitacora)
                            continue
                            
                    
            
    boton_bitacora.click()
    
    try:
        bitacora = driver.find_element_by_xpath('.//div[@id="content-3"]').text
        bitacoras.append(bitacora)
    except:
        
        try:
            if len(bitacora) == 0:
                 bitacora = driver.find_element_by_xpath('.//div[@id="pane-0-1"]').text
            if len(bitacora) == 0:
                 bitacora = driver.find_element_by_xpath('.//div[@id="pane-0-2"]').text
            if len(bitacora) == 0:
                 bitacora = driver.find_element_by_xpath('.//div[@id="pane-0-3"]').text
            if len(bitacora) == 0:
                 bitacora = driver.find_element_by_xpath('.//div[@id="pane-0-4"]').text
            if len(bitacora) == 0:
                 bitacora = driver.find_element_by_xpath('.//div[@id="pane-0-5"]').text
            if len(bitacora) == 0:
                 bitacora = driver.find_element_by_xpath('.//div[@id="pane-0-6"]').text
            bitacoras.append(bitacora)
        except:
            bitacora = ''
            bitacoras.append(bitacora)

    ### Extrae los links
    try:
        url = driver.find_elements_by_xpath('//div[@class="collapse show"]/div/ul//li/a')

        urls = []
        for r in url:
            url2 = r.get_attribute('href')
            urls.append(url2)

        if len(urls) == 0:
            url = driver.find_elements_by_xpath('//div[@class="filtros"]/ul//li/a')

            urls = []
            for r in url:
                url2 = r.get_attribute('href')
                urls.append(url2)

        urls2 = ''
        for url2 in urls:
            urls2 = urls2 + ', ' + url2
        links.append(urls2)
    except:
        urls2 = ''
        links.append(urls2)




    driver.quit()
    sleep(random.randrange(3, 5))

chile['Título']= titulos
chile['Descripción']=descripciones
chile['Público Objetivo']=publicos
chile['Bitacora']=bitacoras
chile['Link proyecto']=urls_proyectos[0:5]
chile['Pdfs']= links

KeyboardInterrupt: 

In [31]:
chile

,Título,Descripción,Público Objetivo,Bitacora,Link proyecto,Pdfs
0,CONVOCATORIA PARA INDEXACIÓN DE REVISTAS CIENT...,La convocatoria tiene como objetivo identifica...,Revistas científicas chilenas editadas en form...,Fecha de inicio Postulaciones: 13 de junio de ...,https://www.conicyt.cl/informacioncientifica/2...,
1,I Concurso IDeA en dos etapas temático en Inoc...,IMPORTANTE\nSólo pueden postular aquellas inic...,Son entidades elegibles para recibir subsidios...,Fecha de inicio: 25 de septiembre 2019\nFecha ...,https://www.conicyt.cl/fondef/2019/11/20/i-con...,", https://www.conicyt.cl/fondef/files/2019/11/..."
2,IV Concurso IDeA en Dos Etapas – Segunda Etapa,IMPORTANTE\nSólo pueden postular aquellas inic...,Son entidades elegibles para recibir subsidios...,-Fecha de inicio del concurso: 20 de noviembre...,https://www.conicyt.cl/fondef/2019/11/20/iv-co...,", https://s3.amazonaws.com/documentos.anid.cl/..."
3,II Concurso IDeA en Dos Etapas temático en Adu...,IMPORTANTE\nATENCIÓN: Sólo pueden postular aqu...,Son entidades elegibles para recibir subsidios...,"Fecha de inicio: lunes 09 de diciembre 2019, 1...",https://www.conicyt.cl/fondef/2019/12/10/ii-co...,", https://www.conicyt.cl/fondef/files/2017/03/..."
4,"Concurso Programa Regional CLIMAT-Amsud, Año 2020",Objetivos del Concurso:\nDesarrollar la colabo...,"Investigadores/as de universidades, unidades y...",Fecha de inicio de Concurso: 1 de Abril 2020\n...,https://www.conicyt.cl/pci/2019/12/climat-amsu...,", https://s3.amazonaws.com/documentos.anid.cl/..."


['https://www.conicyt.cl/informacioncientifica/2017/06/13/convocatoria-para-indexacon-de-revistas-cientificas-en-scielo-chile/',
 'https://www.conicyt.cl/fondef/2019/11/20/i-concurso-idea-en-dos-etapas-tematico-en-inocuidad-y-calidad-alimentaria-segunda-etapa/',
 'https://www.conicyt.cl/fondef/2019/11/20/iv-concurso-idea-en-dos-etapas-segunda-etapa/',
 'https://www.conicyt.cl/fondef/2019/12/10/ii-concurso-idea-en-dos-etapas-tematico-en-adulto-mayor-segunda-etapa/',
 'https://www.conicyt.cl/pci/2019/12/climat-amsud-convocatoria-internacional-2020/']